# Introduction
The main purpose of this notebook is to demonstrate how much money could be made from cross-exchange cryptocurrency trades under the following (very large) assumptions:

- capital controls do not restrict trades
- 0 transaction fees
- trades occur instantaneously
- no limit to the amount of capital that we have

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
single = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\single_merged.csv')

In [3]:
single = single.set_index('timestamp')

## Convert this to a MultiIndex Dataframe

In [4]:
def convert_to_multi(df, timestamp):
    # creating most inner column 
    cols_str = ""
    cols = list(df.columns)
    for elem in cols:
        cols_str = "".join([cols_str, elem]) + " "
    len(cols_str.split())

    # creating the middle column (the exchange: okx, binance_spot, binance_future, binanceus)
    exchanges_str = ""
    for col in cols:
        h, s = col.rfind('-') + 1, col.rfind(':')
        exchanges_str += col[h:s] + " "

    # creating outer column (the feature: volume or vwap)
    features_str = "volume " * int(len(cols_str.split())/2) + "vwap " * int(len(cols_str.split())/2)

    new_df = pd.DataFrame(np.array(df), columns=[features_str.split(), exchanges_str.split(), cols_str.split()]) # this line needs to include a new level
    # new_df.set_index(timestamp, inplace=True)

    d = dict(zip(new_df.columns.levels[2], [word[2][word[2].rfind(':') + 1:] for word in new_df.columns]))
    new_df = new_df.rename(columns=d, level=2)
    new_df = new_df.set_index(timestamp['timestamp'].astype('datetime64[ms]'))
    return new_df

In [14]:
df = convert_to_multi(single, pd.DataFrame(single.index))
df

volume                                        \
                    binance_futures                                         
                           APE_USDT AVAX_USDT AXS_USDT BAKE_USDT BNB_USDT   
timestamp                                                                   
2019-09-08 17:57:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 17:58:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 17:59:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 18:00:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 18:01:00             NaN       NaN      NaN       NaN      NaN   
...                             ...       ...      ...       ...      ...   
2023-01-31 23:55:00             NaN       NaN      NaN       NaN      NaN   
2023-01-31 23:56:00             NaN       NaN      NaN       NaN      NaN   
2023-01-31 23:57:00             NaN       NaN      NaN       NaN      NaN   
2023-01-31 23:58:00             NaN       NaN      NaN       NaN      NaN   
2023-01-31 23:59:00             NaN       NaN      NaN       NaN      NaN   

                                                                   ...  \
                                                                   ...   
                    BTC_BUSD BTC_USDT CRV_USDT CTK_USDT DOGE_USDT  ...   
timestamp                                                          ...   
2019-09-08 17:57:00      NaN    0.001      NaN      NaN       NaN  ...   
2019-09-08 17:58:00      NaN    0.000      NaN      NaN       NaN  ...   
2019-09-08 17:59:00      NaN    0.001      NaN      NaN       NaN  ...   
2019-09-08 18:00:00      NaN    0.000      NaN      NaN       NaN  ...   
2019-09-08 18:01:00      NaN    0.000      NaN      NaN       NaN  ...   
...                      ...      ...      ...      ...       ...  ...   
2023-01-31 23:55:00      NaN      NaN      NaN      NaN       NaN  ...   
2023-01-31 23:56:00      NaN      NaN      NaN      NaN       NaN  ...   
2023-01-31 23:57:00      NaN      NaN      NaN      NaN       NaN  ...   
2023-01-31 23:58:00      NaN      NaN      NaN      NaN       NaN  ...   
2023-01-31 23:59:00      NaN      NaN      NaN      NaN       NaN  ...   

                        vwap                                          \
                         okx                                           
                    FTM_USDT GMT_USDT LINK_USDT MATIC_USDT NEAR_USDT   
timestamp                                                              
2019-09-08 17:57:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 17:58:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 17:59:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 18:00:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 18:01:00      NaN      NaN       NaN        NaN       NaN   
...                      ...      ...       ...        ...       ...   
2023-01-31 23:55:00      NaN      NaN       NaN        NaN       NaN   
2023-01-31 23:56:00      NaN      NaN       NaN        NaN       NaN   
2023-01-31 23:57:00      NaN      NaN       NaN        NaN       NaN   
2023-01-31 23:58:00      NaN      NaN       NaN        NaN       NaN   
2023-01-31 23:59:00      NaN      NaN       NaN        NaN       NaN   

                                                                       
                                                                       
                    SAND_USDT SOL_USDT STORJ_USDT WAVES_USDT XRP_USDT  
timestamp                                                              
2019-09-08 17:57:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 17:58:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 17:59:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 18:00:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 18:01:00       NaN      NaN        NaN        NaN      NaN  
...                       ...    

The above is the correct multiindex dataframe, and the process can be seen in the file `single_df.ipynb`

# Instantaneous Arbitrage Approximation

For each coin, at any time (we can resample later), calculate maximum price dislocation across any exchange. We will buy the lower one and sell the higher one at the maximum volume (which is the lesser of the two volumes). This will be the amount of money we could make at that exact time (second, two seconds, etc). Summing across all time in our dataset, this yields the total amount of money we could make for each asset.

Let us first obtain lists of all assets and all exchanges that we have available in our dataset.

In [8]:
coins = list(df.columns.levels[2])
exchanges = list(df.columns.levels[1])
exchanges

['binance_futures', 'binance_spot', 'binanceus', 'okx']

# Manipulate our MultiIndex Dataframe

Note that we can easily swap the levels of our multiindex dataframe using the `swaplevel()` function.

Observe that `df2` has the following format:
- The outer level represents the security (27 coins on different exchanges)
- The middle level represnts the feature (volume or vwap)
- The inner level represents the exchange (binance_futures, binance_spots, binanceus, okx)

Note that we also don't want to arbitrage any futures right now, so drop all binance_futures

In [16]:
x = df.swaplevel(0, 1, 1).sort_index(axis=1)
x = x[['okx', 'binance_spot', 'binanceus']]
x

okx                                                 \
                      volume                                                  
                    APE_USDT AVAX_USDT AXS_USDT BNB_USDT BTC_USDT DOGE_USDT   
timestamp                                                                     
2019-09-08 17:57:00      NaN       NaN      NaN      NaN      NaN       NaN   
2019-09-08 17:58:00      NaN       NaN      NaN      NaN      NaN       NaN   
2019-09-08 17:59:00      NaN       NaN      NaN      NaN      NaN       NaN   
2019-09-08 18:00:00      NaN       NaN      NaN      NaN      NaN       NaN   
2019-09-08 18:01:00      NaN       NaN      NaN      NaN      NaN       NaN   
...                      ...       ...      ...      ...      ...       ...   
2023-01-31 23:55:00      NaN       NaN      NaN      NaN      NaN       NaN   
2023-01-31 23:56:00      NaN       NaN      NaN      NaN      NaN       NaN   
2023-01-31 23:57:00      NaN       NaN      NaN      NaN      NaN       NaN   
2023-01-31 23:58:00      NaN       NaN      NaN      NaN      NaN       NaN   
2023-01-31 23:59:00      NaN       NaN      NaN      NaN      NaN       NaN   

                                                         ...  binanceus  \
                                                         ...       vwap   
                    DOT_USDT ETH_USDT FTM_USDT GMT_USDT  ...   DOT_USDT   
timestamp                                                ...              
2019-09-08 17:57:00      NaN      NaN      NaN      NaN  ...        NaN   
2019-09-08 17:58:00      NaN      NaN      NaN      NaN  ...        NaN   
2019-09-08 17:59:00      NaN      NaN      NaN      NaN  ...        NaN   
2019-09-08 18:00:00      NaN      NaN      NaN      NaN  ...        NaN   
2019-09-08 18:01:00      NaN      NaN      NaN      NaN  ...        NaN   
...                      ...      ...      ...      ...  ...        ...   
2023-01-31 23:55:00      NaN      NaN      NaN      NaN  ...  14.163121   
2023-01-31 23:56:00      NaN      NaN      NaN      NaN  ...  14.163121   
2023-01-31 23:57:00      NaN      NaN      NaN      NaN  ...  14.163121   
2023-01-31 23:58:00      NaN      NaN      NaN      NaN  ...  14.163121   
2023-01-31 23:59:00      NaN      NaN      NaN      NaN  ...  14.163121   

                                                                              \
                                                                               
                        ETH_BUSD     ETH_USDT  FTM_USDT  LINK_USDT NEAR_USDT   
timestamp                                                                      
2019-09-08 17:57:00          NaN          NaN       NaN        NaN       NaN   
2019-09-08 17:58:00          NaN          NaN       NaN        NaN       NaN   
2019-09-08 17:59:00          NaN          NaN       NaN        NaN       NaN   
2019-09-08 18:00:00          NaN          NaN       NaN        NaN       NaN   
2019-09-08 18:01:00          NaN          NaN       NaN        NaN       NaN   
...                          ...          ...       ...        ...       ...   
2023-01-31 23:55:00  2373.882896  2143.995885  0.975800  10.016476  8.139902   
2023-01-31 23:56:00  2373.882316  2143.993804  0.975799  10.016476  8.139902   
2023-01-31 23:57:00  2373.882316  2143.992917  0.975799  10.016476  8.139902   
2023-01-31 23:58:00  2373.882211  2143.990863  0.975797  10.016476  8.139902   
2023-01-31 23:59:00  2373.882211  2143.989782  0.975797  10.016476  8.139902   

                                                              
                                                              
                     OGN_USDT SAND_USDT   SOL_USDT  XRP_USDT  
timestamp                                                     
2019-09-08 17:57:00       NaN       NaN        NaN       NaN  
2019-09-08 17:58:00       NaN       NaN        NaN       NaN  
2019-09-08 17:59:00       NaN       NaN        NaN       NaN  
2019-09-08 18:00:00       NaN       NaN        NaN       NaN  
2019-09-

In [17]:
df2 = x.swaplevel(0, 2, 1).sort_index(axis=1)
df2

APE_USDT                                           \
                          volume                       vwap                 
                    binance_spot binanceus okx binance_spot binanceus okx   
timestamp                                                                   
2019-09-08 17:57:00          NaN       NaN NaN          NaN       NaN NaN   
2019-09-08 17:58:00          NaN       NaN NaN          NaN       NaN NaN   
2019-09-08 17:59:00          NaN       NaN NaN          NaN       NaN NaN   
2019-09-08 18:00:00          NaN       NaN NaN          NaN       NaN NaN   
2019-09-08 18:01:00          NaN       NaN NaN          NaN       NaN NaN   
...                          ...       ...  ..          ...       ...  ..   
2023-01-31 23:55:00       154.52       0.0 NaN     9.411818   9.78506 NaN   
2023-01-31 23:56:00       264.84       0.0 NaN     9.411818   9.78506 NaN   
2023-01-31 23:57:00       287.58       0.0 NaN     9.411818   9.78506 NaN   
2023-01-31 23:58:00       887.80       0.0 NaN     9.411818   9.78506 NaN   
2023-01-31 23:59:00       404.81       0.0 NaN     9.411817   9.78506 NaN   

                       AVAX_USDT                             ...   WAVES_USDT  \
                          volume                       vwap  ...       volume   
                    binance_spot binanceus okx binance_spot  ... binance_spot   
timestamp                                                    ...                
2019-09-08 17:57:00          NaN       NaN NaN          NaN  ...          NaN   
2019-09-08 17:58:00          NaN       NaN NaN          NaN  ...          NaN   
2019-09-08 17:59:00          NaN       NaN NaN          NaN  ...          NaN   
2019-09-08 18:00:00          NaN       NaN NaN          NaN  ...          NaN   
2019-09-08 18:01:00          NaN       NaN NaN          NaN  ...          NaN   
...                          ...       ...  ..          ...  ...          ...   
2023-01-31 23:55:00       599.82       0.0 NaN    43.936134  ...       281.75   
2023-01-31 23:56:00       584.41       0.0 NaN    43.936128  ...       976.73   
2023-01-31 23:57:00      1333.10       0.0 NaN    43.936115  ...       227.68   
2023-01-31 23:58:00       642.07       0.0 NaN    43.936109  ...      8895.36   
2023-01-31 23:59:00       304.73       0.0 NaN    43.936106  ...      1272.68   

                                             XRP_USDT                \
                                vwap           volume                 
                    okx binance_spot okx binance_spot binanceus okx   
timestamp                                                             
2019-09-08 17:57:00 NaN          NaN NaN          NaN       NaN NaN   
2019-09-08 17:58:00 NaN          NaN NaN          NaN       NaN NaN   
2019-09-08 17:59:00 NaN          NaN NaN          NaN       NaN NaN   
2019-09-08 18:00:00 NaN          NaN NaN          NaN       NaN NaN   
2019-09-08 18:01:00 NaN          NaN NaN          NaN       NaN NaN   
...                  ..          ...  ..          ...       ...  ..   
2023-01-31 23:55:00 NaN    10.713475 NaN     135505.0   11914.6 NaN   
2023-01-31 23:56:00 NaN    10.713472 NaN     177663.0   11914.6 NaN   
2023-01-31 23:57:00 NaN    10.713472 NaN     197800.0   11914.6 NaN   
2023-01-31 23:58:00 NaN    10.713444 NaN     159875.0   11914.6 NaN   
2023-01-31 23:59:00 NaN    10.713440 NaN      73224.0   11914.6 NaN   

                                                
                            vwap                
                    binance_spot binanceus okx  
timestamp                                       
2019-09-08 17:57:00          NaN       NaN NaN  
2019-09-08 17:58:00          NaN       NaN NaN  
2019-09-08 17:59:00          NaN       NaN NaN  
2019-09-08 18:00:00          NaN       NaN NaN  
2019-09-08 18:01:00          NaN       NaN NaN  
...                          ...       ...  ..  
2023-01-31 23:55:00     0.735969  0.275762 NaN  
2023-01-31 23:56:00     0.735969  0.275762 NaN 

In [30]:
coins = list(df2.columns.levels[0])
exchanges = list(df2.columns.levels[2])
exchanges.pop(0)
exchanges

['binance_spot', 'binanceus', 'okx']

NaN values simply mean that the crypto coin was not tradeable on some exchange at that time.

Let's develop this algorithm for 1 coin for now, say Bitcoin. Now let us denote $\mu =$ instance profit.

$$\mu = \min(volume_i, volume_j) * (price_j - price_i)$$

                  price_i        price_j          i                j              volume[i]     volume[j]      instance_profit 
        9:30        10              8         binanceus     binance_futures           2            5              
        9:31        12              8            okx        binance_futures           3            7

where $i, j$ are the same coin but on different exchanges. They represent different indices in the dataframe.

Let $price_i$ be the minimum price of some arbitrary coin and let $price_j$ be the maximum price of the same arbirary coin at some time $t$

First, let us get all of our needed columns so we can achieve all calculations vectorially and therefore exponentially faster.

## DataFrame Creation

In [31]:
btc = pd.DataFrame(df2.index)
btc = btc.set_index('timestamp')
btc['price_i'] = df2['BTC_USDT']['vwap'][exchanges].min(axis=1)
btc['price_j'] = df2['BTC_USDT']['vwap'][exchanges].max(axis=1)
btc['i'] = df2['BTC_USDT']['vwap'][exchanges].idxmin(axis=1)
btc['j'] = df2['BTC_USDT']['vwap'][exchanges].idxmax(axis=1)
btc['binance_spot'] = df2['BTC_USDT']['volume']['binance_spot']
btc['binanceus'] = df2['BTC_USDT']['volume']['binanceus']
btc['okx'] = df2['BTC_USDT']['volume']['okx']
btc

,price_i,price_j,i,j,binance_spot,binanceus,okx
timestamp,,,,,,,
2019-09-08 17:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 17:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 17:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-01-31 23:55:00,27207.485001,28969.897150,binanceus,binance_spot,132.98878,0.665210,NaN
2023-01-31 23:56:00,27207.471198,28969.884672,binanceus,binance_spot,186.18800,2.498794,NaN
2023-01-31 23:57:00,27207.461297,28969.877264,binanceus,binance_spot,110.49868,1.791509,NaN


## Optimizing Calculations

Let's see how many of these rows are even useful (ie. $i \neq j \implies \ \exists$ a price dislocation)

In [35]:
btc2 = btc.loc[btc['i'] != btc['j']] # only keep times where the the lowest price != maximum price
btc2 = btc2.dropna(subset=['i','j'])
display(btc2)
print("Percentage of minutes where there exists some price dislocation for Bitcoin", len(btc2)/len(df2))

,price_i,price_j,i,j,binance_spot,binanceus,okx
timestamp,,,,,,,
2021-01-01 00:00:00,28937.390000,28943.335000,binance_spot,binanceus,27.457032,0.138756,NaN
2021-01-01 00:01:00,28972.683560,28986.477367,binance_spot,binanceus,58.477501,1.543793,NaN
2021-01-01 00:02:00,28980.112755,28986.958822,binance_spot,binanceus,42.470329,0.094843,NaN
2021-01-01 00:03:00,28981.255781,28986.846904,binance_spot,binanceus,30.360677,0.042083,NaN
2021-01-01 00:04:00,28981.599954,28986.856765,binance_spot,binanceus,24.124339,0.019755,NaN
...,...,...,...,...,...,...,...
2023-01-31 23:55:00,27207.485001,28969.897150,binanceus,binance_spot,132.988780,0.665210,NaN
2023-01-31 23:56:00,27207.471198,28969.884672,binanceus,binance_spot,186.188000,2.498794,NaN
2023-01-31 23:57:00,27207.461297,28969.877264,binanceus,binance_spot,110.498680,1.791509,NaN


Percentage of minutes where there exists some price dislocation for Bitcoin 0.6122344050191283


## Calculating the maximum amount of tradeable volume

We can achieve our goal by grouping the dataframe into smaller dataframes by $i, j$ pairs such that we can perform only vectorial operations.

In [36]:
grouped = btc2.groupby(['i', 'j'])
dfs = [grouped.get_group(x) for x in grouped.groups]
dfs[0]

,price_i,price_j,i,j,binance_spot,binanceus,okx
timestamp,,,,,,,
2021-01-01 00:00:00,28937.390000,28943.335000,binance_spot,binanceus,27.457032,0.138756,NaN
2021-01-01 00:01:00,28972.683560,28986.477367,binance_spot,binanceus,58.477501,1.543793,NaN
2021-01-01 00:02:00,28980.112755,28986.958822,binance_spot,binanceus,42.470329,0.094843,NaN
2021-01-01 00:03:00,28981.255781,28986.846904,binance_spot,binanceus,30.360677,0.042083,NaN
2021-01-01 00:04:00,28981.599954,28986.856765,binance_spot,binanceus,24.124339,0.019755,NaN
...,...,...,...,...,...,...,...
2021-05-23 16:59:00,46065.208969,46067.554874,binance_spot,binanceus,475.509555,7.456140,NaN
2021-05-23 17:00:00,46064.597332,46066.460288,binance_spot,binanceus,564.063912,6.514261,NaN
2021-05-23 17:01:00,46064.275062,46065.989485,binance_spot,binanceus,301.977989,2.847467,NaN


Perfom the calculation for the minimum of the $i, j$ volumes for each dataframe in the list of daataframes.

In [37]:
for df in dfs:
    exchange1, exchange2 = df['i'].unique()[0], df['j'].unique()[0]
    df['min_volume'] = df[[exchange1, exchange2]].min(axis=1)
dfs[1] # example df grouped by i, j

,price_i,price_j,i,j,binance_spot,binanceus,okx,min_volume
timestamp,,,,,,,,
2021-01-01 15:00:00,29238.913472,29319.650000,binance_spot,okx,30.950892,0.003438,2.567870,2.567870
2021-01-01 15:01:00,29238.968363,29314.678723,binance_spot,okx,28.580070,0.191991,1.985712,1.985712
2021-01-01 15:02:00,29239.011490,29282.948554,binance_spot,okx,38.118284,0.025966,35.688282,35.688282
2021-01-01 15:03:00,29239.062360,29281.927038,binance_spot,okx,55.091686,0.082348,4.360614,4.360614
2021-01-01 15:04:00,29239.081747,29281.734065,binance_spot,okx,17.684160,0.000000,2.620863,2.620863
2021-01-01 15:05:00,29239.148544,29282.707190,binance_spot,okx,43.644885,0.171334,4.358821,4.358821
2021-01-01 15:06:00,29239.257245,29285.881850,binance_spot,okx,47.510166,0.008636,4.435645,4.435645
2021-01-01 15:07:00,29239.325019,29288.207430,binance_spot,okx,28.610033,0.013254,3.722902,3.722902
2021-01-01 15:08:00,29239.392571,29290.537602,binance_spot,okx,29.387381,0.006898,4.532570,4.532570


Concatenate the dataframes and then sort by the timestamp.

In [39]:
final_btc = pd.concat(dfs)
final_btc = final_btc.sort_index()
final_btc = final_btc.drop(columns=['binance_spot', 'binanceus', 'okx'])
final_btc

,price_i,price_j,i,j,min_volume
timestamp,,,,,
2021-01-01 00:00:00,28937.390000,28943.335000,binance_spot,binanceus,0.138756
2021-01-01 00:01:00,28972.683560,28986.477367,binance_spot,binanceus,1.543793
2021-01-01 00:02:00,28980.112755,28986.958822,binance_spot,binanceus,0.094843
2021-01-01 00:03:00,28981.255781,28986.846904,binance_spot,binanceus,0.042083
2021-01-01 00:04:00,28981.599954,28986.856765,binance_spot,binanceus,0.019755
...,...,...,...,...,...
2023-01-31 23:55:00,27207.485001,28969.897150,binanceus,binance_spot,0.665210
2023-01-31 23:56:00,27207.471198,28969.884672,binanceus,binance_spot,2.498794
2023-01-31 23:57:00,27207.461297,28969.877264,binanceus,binance_spot,1.791509


Finally we have obtained a complete df our desired format, and so now we can easily recompute the instant profits as the following:
$$\mu = \min(volume_i, volume_j) * (price_j - price_i)$$

In [40]:
final_btc['instant_profit'] = final_btc['min_volume'] * abs(final_btc['price_i'] - final_btc['price_j'])
final_btc

,price_i,price_j,i,j,min_volume,instant_profit
timestamp,,,,,,
2021-01-01 00:00:00,28937.390000,28943.335000,binance_spot,binanceus,0.138756,0.824904
2021-01-01 00:01:00,28972.683560,28986.477367,binance_spot,binanceus,1.543793,21.294783
2021-01-01 00:02:00,28980.112755,28986.958822,binance_spot,binanceus,0.094843,0.649302
2021-01-01 00:03:00,28981.255781,28986.846904,binance_spot,binanceus,0.042083,0.235291
2021-01-01 00:04:00,28981.599954,28986.856765,binance_spot,binanceus,0.019755,0.103848
...,...,...,...,...,...,...
2023-01-31 23:55:00,27207.485001,28969.897150,binanceus,binance_spot,0.665210,1172.374186
2023-01-31 23:56:00,27207.471198,28969.884672,binanceus,binance_spot,2.498794,4403.908214
2023-01-31 23:57:00,27207.461297,28969.877264,binanceus,binance_spot,1.791509,3157.384066


In [41]:
final_btc['instant_profit'].sum()

1524393462.4074972

# Extending The Algorithm For All Coins

For now, I think we can represent this as a hashmap of dataframes such that the key is the coin and the value is the dataframe that we described above.

In [72]:
def calculate_immediate_arb(coin_df):
    
    # get all necessary columns for vectorial operations
    coin = pd.DataFrame(coin_df.index)
    coin = coin.set_index('timestamp')
    exchange_list = list(coin_df['vwap'].columns)

    coin['price_i'] = coin_df['vwap'][exchange_list].min(axis=1)
    coin['price_j'] = coin_df['vwap'][exchange_list].max(axis=1)
    coin['i'] = coin_df['vwap'][exchange_list].idxmin(axis=1)
    coin['j'] = coin_df['vwap'][exchange_list].idxmax(axis=1)
    for exchange in exchange_list:
        coin[exchange] = coin_df['volume'][exchange]

    # only consider rows where there exists a price dislocation and i != j != NaN
    coin2 = coin.loc[coin['i'] != coin['j']]
    coin2 = coin2.dropna(subset=['i','j'])

    # group by i, j pairs
    grouped = coin2.groupby(['i', 'j'])
    dfs = [grouped.get_group(x) for x in grouped.groups]
    
    # compute the min_volume column for each groupedcoin_df 
    for x in dfs:
        exchange1, exchange2 = x['i'].unique()[0], x['j'].unique()[0]
        x['min_volume'] = x[[exchange1, exchange2]].min(axis=1)
    
    # concatenate them, sort by tiemstamp, and drop unnecessary columns
    if dfs:
        final_coin = pd.concat(dfs)
        final_coin = final_coin.sort_index()
        final_coin = final_coin.drop(columns=exchange_list)
        final_coin['instant_profit'] = final_coin['min_volume'] * abs(final_coin['price_i'] - final_coin['price_j'])
    else:
        final_coin = coin
        final_coin['instant_profit'] = np.zeros(len(coin_df))
    return final_coin

In [73]:
immediate_arb = {}
for coin in coins:
    immediate_arb[coin] = calculate_immediate_arb(df2[coin])

In [74]:
immediate_arb['BTC_USDT']['instant_profit'].sum() #verifying that the helper function worked as planned 

1524393462.4074972

In [75]:
for x in immediate_arb.keys():
    print(x, immediate_arb[x]['instant_profit'].sum())

APE_USDT 72915193.82981107
AVAX_USDT 398181418.26193786
AXS_USDT 431698080.6441472
BAKE_USDT 0.0
BNB_USDT 86683390.21636243
BTC_BUSD 274172569.32295734
BTC_USDT 1524393462.4074972
CRV_USDT 66676391.92813693
CTK_USDT 0.0
DOGE_USDT 351913613.7159651
DOT_USDT 190419467.93767932
DYDX_USDT 0.0
ETH_BUSD 125567699.12481526
ETH_USDT 421988296.23561335
FTM_USDT 658468734.6856314
GMT_USDT 205651347.68731874
LINK_USDT 97544900.39348477
MATIC_USDT 1254768195.3645854
NEAR_USDT 85434451.85390842
OGN_USDT 48821661.45151767
RUNE_USDT 0.0
SAND_USDT 146899506.88895097
SOL_USDT 508877626.5179783
STORJ_USDT 23017885.46719864
UNFI_USDT 0.0
WAVES_USDT 112523161.77233344
XRP_USDT 6013211484.861705
